In [1]:
from __future__ import absolute_import, division, print_function

from os import listdir
import xmltodict
import pandas
from StringIO import StringIO
%matplotlib inline
from tqdm import tqdm
import math
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from datetime import date
from praelexis_file_system_provider.data_access_layer import AzureBlobStorageDataAccessLayer
import PyPDF2
import subprocess
import shutil
import os

from PIL import Image

import re
import glob
import pandas as pd

### Test Regular Expression

Exact match

In [ ]:
pattern = r"Cookie"
sequence = "Cookie"
if re.match(pattern, sequence):
    print("Match!")
else: 
    print("Not a match!")

Search for string

In [ ]:
m = re.search('(?<=abc)def', 'abcdef')
print(m.group(0))

Match string with some characters general

In [ ]:
m = re.search(r'Co.k.e', 'Cookie')
print(m.group(0))

### Print Columns

In [ ]:
line = "| 10-28 | DEBIT CARD PURCHASE FROM SUPER 6 TO R-748.03 R 968.76"
line = "09-30 SBSA-FUNERAL PLAN R-130.00 R-208. 54"

line = "09-30 SBSA-FUNERAL PLAN R -130 .00 R-208 54"
line = "10-31 FEE- POS DECLINED INSUFF FUNDS 16H47 R-7.90 RT."
replaced = re.sub(' R ', ' R', line)
replaced = re.sub('\|', '', replaced)
replaced = re.sub('  ', ' ', replaced)
replaced = re.sub('(?<=\d\.)\s(?=\d)', '', replaced)
replaced = re.sub('(?<=\d)\s(?=\.\d)', '', replaced)

values = re.finditer(r'\sR[0-9,\-,\s][0-9,.][0-9,.,-]{0,10}',replaced)
for index,value in enumerate(values):
    print(index," ",value.group())

In [42]:
import glob
folderLoc = "./test_files/"
txtNames = glob.glob(folderLoc+"*.txt")
print("Number of textfiles found: ",len(txtNames))
result = np.zeros(len(txtNames))
#for i in tqdm(range(len(txtNames))):
print(txtNames[0])
f = open(txtNames[0],'r')
values = []
line = f.readline()
while line: 
    print(line)
    line = f.readline()
    

Number of textfiles found:  4000
./test_files/Example1000.txt
 



 



 



ACCESSACC

§ Standard Bank

3 months

STANDARD BANK Stain

FOLOKWANE

SQUARE BRANCH

18 Dec 2017

7182



 



 



Azcount number

Statament period



Account preferred centre

Customer Contact Centre

manager

Telephone



e-mail



Internet



Page



Date

Time



 



10 052 577 390

18 December 2017 to

26 September 2017



0860 101 341



wi, standardbank. co. 2a

lof 8



18 December 2017

08:06:22



Address : WARD 20 70S TSHAMAHANST VILLAG



 



 



 



 



 



 



Account: identification MOKOPANE

Name of account: MR FA KHUTWANE 0600

Account number: 10 052 577 390

Transaction details

Post

| Nii Transaction description Fee Payments Deposits Balance

mn

Statement opening balances R 3.36

09-26 | FEE- POS DECLINED INSUFF FUNDS O7H34 R-7.90 R-4.54

payD MTN Eazi Recharge Jo

09-29 | SALARY RUVIC SALARY R 7,616.26 R 7,611.72

09-29 | NAEDO REPRESENTATION OLD R-80.56 R 7,531.16

MUTUAL667P0392

### Generate columns from the text

In [5]:
def pre_process_line(line):
    # Pre Process text
    replaced = re.sub(' R ', ' R', line)
    replaced = re.sub('\|', '', replaced)
    replaced = re.sub('  ', ' ', replaced)
    replaced = re.sub('(?<=\d\.)\s(?=\d)', '', replaced)
    replaced = re.sub('(?<=\d)\s(?=\.\d)', '', replaced)
    line = re.sub('\n', '', replaced)
    return line

def create_dict():
    # Create Dictionary
    columns = {}
    columns["Date"] = "NONE"
    columns["Description"] = "NONE"
    columns["Payment"] = "NONE"
    columns["Deposit"] = "NONE"
    columns["Balance"] = "NONE"
    columns["Unknown"] = "NONE"
    return columns

def find_date(line,columns):
    # Find the date
    date = re.search(r'\d\d-\d\d\s', line)
    if date:
        columns["Date"] = date.group()
    return date 

def find_amount_values(line,date):
    readStart = 0
    if date:
        readStart = date.end()
        values = re.finditer(r'\sR[0-9,\-][0-9,.][0-9,.,-,]{0,10}',line[date.end():])
    else:
        values = re.finditer(r'\sR[0-9,\-][0-9,.][0-9,.,-,]{0,10}',line)
    return values, readStart
    

In [35]:
def process_line(line):
    line = pre_process_line(line)
    columns = create_dict()
    date = find_date(line,columns)
    values, readStart = find_amount_values(line,date)
        
    minValues = len(line)
    
    list_values  = [value for value in values]
    
    if values:
        for index,value in enumerate(list_values):
            if value.start()<minValues:
                minValues = value.start()+readStart
            replaced = re.sub('R', '', str(value.group()))
            replaced = re.sub(' ', '', str(replaced))
            if len(replaced) > 3:
                i = len(replaced)-3
                lower = re.sub(',', '', replaced[:i])
                lower = re.sub('\.', '', lower)
                upper = re.sub(',', '.', replaced[i:])
                
                replaced = lower + upper
            
#             if index==0:
#                 columns["Payment"] = replaced
#             else:
#                 columns["Balance"] = replaced
                    
            if len(list_values)==2:
                if index==0:
                    #print("1 '",replaced,"'")
                    #print(replaced.replace('.','',1).replace('-','',1).isdigit()) 
                    if replaced.replace('.','',1).replace('-','',1).isdigit():

                        float_value = float(replaced)
                        
                        if float_value > 0.0:
                            columns["Deposit"] = replaced
                        else:
                            columns["Payment"] = replaced
                else:
                    #print("2 '",replaced,"'")
                    columns["Balance"] = replaced
            elif len(list_values)==1:
                print("here")
                check1 = "statement" in line.lower()
                check2 = "opening" in line.lower()
                check3 = "balance" in line.lower()
                check4 = "brought" in line.lower()
                check5 = "forward" in line.lower()
                print(line.lower(), " ", check1, " ", check2, " ", check3, " ", check4, " ", check5)
                if check1 or check2 or check3 or check4 or check5:
                    print(replaced)
                    columns["Balance"] = replaced
                

    if date and values:
        columns["Description"] = line[date.end():minValues]
    elif values:
        columns["Description"] = line[:minValues]
    elif date:
        columns["Description"] = line[date.end():]
    else:
        columns["Description"] = line
    
    return columns

In [21]:
def get_columns(txtFile):
    # Read in data
    f = open(txtFile,'r')
    # Create DataFrame
    data = pd.DataFrame(columns=['Date','Description','Payment','Balance'])
    line = f.readline()
    
    while line:
        columns = process_line(line)
        #print(line)
        
        
        if len(data)>0:
            preLineTest = np.all(data.tail(1)!="NONE")
        else:
            preLineTest = False
        curLineTest = (columns["Date"] == "NONE" and columns["Description"] != "NONE" and columns["Payment"] == "NONE" and columns["Balance"] == "NONE")
        
        #if preLineTest:
            #print("PreLine passed.")
        
        if preLineTest and curLineTest:
            data.tail(1)["Description"] += " " + columns["Description"]
            #print("Concatenate: ", data.tail(1)," ",columns)
            #print(preLineTest)
        elif columns["Payment"] != "NONE" or columns["Balance"] != "NONE": #Only add reconds that has non zero
            #print("Don't concatenate: ",preLineTest," ",curLineTest)
            data = data.append(columns,ignore_index=True)
        
        
        line = f.readline()
    return data

In [36]:

folderLoc = "./test_files/"
txtNames = glob.glob(folderLoc+"*.txt")
print("Number of textfiles found: ",len(txtNames))
result = np.zeros(len(txtNames))
#for i in tqdm(range(len(txtNames))):
df = get_columns(txtNames[0])


Number of textfiles found:  4000
here
statement opening balances r3.36   True   True   True   False   False
3.36
here
balance brought forward r-208.54   False   False   True   True   True
-208.54
here
 10-27 naedo representation old r-80.56 7,534.44   False   False   False   False   False
here
 balance brought forward r7,524.94   False   False   True   True   True
7524.94
here
10-31 fee- pos declined insuff funds 16h47 r-7.90 rt.   False   False   False   False   False
here
 balance brought forward r-27.6%   False   False   True   True   True
-27.6
here
u2 ser a at chet it re450.00 r3,507.62   False   False   False   False   False
here
balance brought forward r350.73   False   False   True   True   True
350.73
here
balance brought forward r1,520.39   False   False   True   True   True
1520.39
here
 deel cand sos rl, 158.32 r7,864.58   False   False   False   False   False
here
1-18 oer caso pron esas re434.60 r7,429.98   False   False   False   False   False
here
paymen’s r-44,007.40  

In [41]:
df.iloc[60:80]

,Date,Description,Payment,Balance,Deposit,Unknown
60,10-28,AUTOBANK CASH WITHDRAWAL AT STDACT31,-600.00,368.76,NONE,NONE
61,10-28,CASH WITHDRAWAL FEE 10052577390,-9.60,359.16,NONE,NONE
62,NONE,028 EET EAR 80s,-189.00,170.16,NONE,NONE
63,NONE,[0-28 DEBLT CARD PURCHASE FROM P2YD,-12.00,158.16,NONE,NONE
64,NONE,{0-31 DESLT CARD PURCHASE FRom pafD,-12.00,146.16,NONE,NONE
65,NONE,130 lige conn enon,-12.00,134.16,NONE,NONE
66,10-31,A Ry FROM POLITSI,-114.00,20.16,NONE,NONE
67,10-31,SBSA-FUNERAL PLAN,-130.00,-109.84,NONE,NONE
68,NONE,L103 RTD-NOT PROVIDED FOR,NONE,20.16,130.00,NONE
69,10-31,MONTHLY ACCOUNT FEE,-4.99,15.17,NONE,NONE


### Plot the results

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
x = result#data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAABJIAAAFHCAYAAAD6JmYkAAAABHNCSVQICAgIfAhkiAAAAAlwSFlzAAALEgAACxIB0t1+/AAAHOxJREFUeJzt3Xu0pWddH/DvLxkIuTADiM6YBBMgUFQMCA2yhNIRkJsItmgVVCSuKqtVQIsWse3KLJe3ZZetSMQajKkokYpxlWjFJFwGGyFibgQlXFQSEZyhMRBuIYTw6x9nJxyGzJxnzsw+79n7fD5rnZW93/Nefk/e8+7Ld57neau7AwAAAABrOWbqAgAAAABYDIIkAAAAAIYIkgAAAAAYIkgCAAAAYIggCQAAAIAhgiQAAAAAhsw1SKqq86tqf1Vdt2rZL1XV9VV1bVVdVFXb51kDAAAAAEfHvHskXZDkqQcsuzTJ13f3I5N8IMnL51wDAAAAAEfBXIOk7r48yccOWPam7v7C7OkVSU6dZw0AAAAAHB1Tz5H0g0neOHENAAAAAAyYLEiqqv+U5PbuvnCqGgAAAAAYt22Kg1bVC5I8I8kT11ivN6QgAAAAgC2ku2s9221EkFSzn5UnVU9L8pNJntDdt621cbcsCTbanj17smfPnqnLgC3HtQfTcf3BNFx7MI2qdWVISeY8tK2qLkzy9iQPraq/r6qzk7wyyUlJLquqq6vqVfOsAQAAAICjY649krr7eXez+IJ5HhMAAACA+Zj6rm3AJrR79+6pS4AtybUH03H9wTRce7B4ajPPQVRVvZnrAwAAAFg0VbXuybb1SAIAAABgiCAJAAAAgCGCJAAAAACGCJIAAAAAGCJIAgAAAGCIIAkAAACAIYIkAAAAAIYIkgAAAAAYIkgCAAAAYIggCQAAAIAhgiQAAAAAhgiSAAAAABgiSAIAAABgiCAJAAAAgCGCJAAAAACGCJIAAAAAGCJIAgAAAGDItqkLAAAAYGvatev07N9/49RlzM3Onadl374bpi4Djqrq7qlrOKiq6s1cHwAAAOtXVUmW+TtfxXdaNqOqSnfXerY1tA0AAACAIYIkAAAAAIYIkgAAAAAYIkgCAAAAYIggCQAAAIAhgiQAAAAAhgiSAAAAABgiSAIAAABgiCAJAAAAgCGCJAAAAACGCJIAAAAAGCJIAgAAAGCIIAkAAACAIYIkAAAAAIYIkgAAAAAYIkgCAAAAYIggCQAAAIAhcw2Squr8qtpfVdetWnbfqrq0qt5XVZdU1Y551gAAAADA0THvHkkXJHnqAct+KsmbuvufJXlLkpfPuQYAAAAAjoK5BkndfXmSjx2w+NlJfnv2+LeTfMc8awAAAADg6JhijqSv6u79SdLd+5J81QQ1AAAAAHCYtk1dQJI+1C/37Nlz1+Pdu3dn9+7dcy4HAAAAYHns3bs3e/fuPSr7qu5D5jhHfoCq05L8UXefOXt+fZLd3b2/qnYleWt3f+1Btu151wcAAMA0qipr9C1YcBXfadmMqirdXevZdiOGttXs504XJ3nB7PEPJHnDBtQAAAAAwBGaa4+kqrowye4kX5Fkf5JzkvzvJK9P8oAkNyb5N9398YNsr0cSAADAktIjCaZxJD2S5j607UgIkgAAAJaXIAmmsdmHtgEAAACwBARJAAAAAAwRJAEAAAAwRJAEAAAAwBBBEgAAAABDBEkAAAAADBEkAQAAADBEkAQAAADAEEESAAAAAEMESQAAAAAMESQBAAAAMESQBAAAAMAQQRIAAAAAQwRJAAAAAAwRJAEAAAAwRJAEAAAAwBBBEgAAAABDBEkAAAAADBEkAQAAADBk29QFAAAAAItl167Ts3//jVOXMTc7d56WfftumLqMTam6e+oaDqqqejPXBwAAwPpVVZJl/s5XWdbvtM7dYquqdHetZ1tD2wAAAAAYIkgCAAAAYIggCQAAAIAhgiQAAAAAhgiSAAAAABgiSAIAAABgiCAJAAAAgCGCJAAAAACGCJIAAAAAGCJIAgAAAGCIIAkAAACAIYIkAAAAAIYIkgAAAAAYIkgCAAAAYIggCQAAAIAhgiQAAAAAhkwWJFXVj1fVX1XVdVX12qq651S1AAAAALC2SYKkqjo5yYuSPKq7z0yyLcn3TFELAAAAAGO2TXjsY5OcWFVfSHJCko9MWAsAAAAAa5ikR1J3fyTJLyf5+yQfTvLx7n7TFLUAAAAAMGaqoW33SfLsJKclOTnJSVX1vClqAQAAAGDMVEPbnpzk77r75iSpqj9M8s1JLjxwxT179tz1ePfu3dm9e/fGVAgAwF127To9+/ffOHUZc7Nz52nZt++GqctgHfxtAqxt79692bt371HZV3X3UdnRYR206jFJzk9yVpLbklyQ5C+7+9cOWK+nqA8AgC9VVUmW+XNZxefOxeRvc7E5f4vLuVtsVZXurvVsO9UcSe9M8gdJrknyriSV5LwpagEAAABgzCQ9kkbpkQQAsDn4l2c2K3+bi835W1zO3WJbuB5JAAAAACweQRIAAAAAQwRJAAAAAAwRJAEAAAAwRJAEAAAAwBBBEgAAAABDBEkAAAAADBEkAQAAADBEkAQAAADAEEESAAAAAEMESQAAAAAMESQBAAAAMESQBAAAAMAQQRIAAAAAQwRJAAAAAAwRJAEAAAAwZChIqqpfqqrtVXWPqnpzVf2/qvq+eRcHAAAAwOYx2iPpKd39iSTPTHJDkjOS/OS8igIAAABg8xkNku4x+++3JXl9d98yp3oAAAAA2KS2Da73R1X13iS3Jvl3VfWVST47v7K2hl27Ts/+/TdOXcbc7Nx5Wvbtu2HqMgDYJLzvwTSW/dpbfselqqYuAuAu1d1rr1R1XJITk9zS3XdU1YlJTuru/XMtrqpH6ltUK28Iy9u+pLLM5w+Aw+N9b7E5f4trK5w77Vtky98+ry2LannPXbJy/rp7XSn16NC2d3T3zd19R5J096eTvHE9BwQAAABgMR1yaFtV7UpySpLjq+obsxIXJ8n2JCfMuTYAAAAANpG15kh6apIXJDk1yX9btfyTSX56TjUBAAAAsAmNzpH0nO6+aAPqOfC45khaaMs9phSAw+N9b7E5f4trK5w77Vtky98+ry2LannPXXJkcyQdzmTbz0lyelb1Yurun1nPQUcJkhbdcl94ABwe73uLzflbXFvh3GnfIlv+9nltWVTLe+6SIwuS1hradqc3JLklyVVJblvPgQAAAABYbKNB0qnd/bS5VgIAAADApnbM4Hpvr6pvmGslAAAAAGxqo3MkvSfJGUk+mJWhbZWku/vMuRZnjqQFt9xjSgE4PN73Fpvzt7i2wrnTvkW2/O3z2rKolvfcJRszR9LT17NzAAAAAJbH0NC27r4xyQOSPHH2+DOj2wIAAACwHIbCoKo6J8nLkrx8tugeSX53XkUBAAAAsPmM9ir6V0meleTTSdLdH0ly73kVBQAAAMDmMxokfW4263UnSVWdOL+SAAAAANiMRoOk36+q30hyn6r6oSRvSvLq+ZUFAAAAwGZTo7ezq6pvTfKUrNyf8ZLuvmyehc2O2ct+uz23SwRgq/C+t9icv8W1Fc6d9i2y5W+f15ZFtbznLlk5f91d69r2cP7HVNX2JNvufN7dN6/noIdxPEHSQlvuCw+Aw+N9b7E5f4trK5w77Vtky98+ry2LannPXXJkQdLoXdteWFX7klyX5MokV83+u25VtaOqXl9V11fVX1fVNx3J/gAAAACYr21rr5Ik+YkkD+/um47isV+R5E+6+7uqaluSE47ivgEAAAA4ykaDpL9N8pmjddDZELl/0d0vSJLu/nySTxyt/QMAAABw9I0GSS9P8vaq+oskt925sLtfvM7jPjDJTVV1QZJHZGWY3Eu6+9YDV7zqqqvWeYjN7bjjjpu6BAAAAIDDMhok/UaStyR5d5IvHKXjPirJj3T3lVX1K0l+Ksk5B674uMc984sbbbt3tm2791E4/PRuvfX9U5cAAAAAbAF79+7N3r17j8q+hu7aVlXXdPc3HpUjruxvZ5J3dPeDZs8fn+Rl3f3tB6zXyzoL/I4du3PLLW/LsrZvxXLPcg/A4XF3l8Xm/C2urXDutG+RLX/7vLYsquU9d8kG3LUtyRur6oer6qur6n53/qzngEnS3fuTfKiqHjpb9KQk71nv/gAAAACYv9Ghbc+d/fflq5Z1kgcdwbFfnOS1VXWPJH+X5Owj2BcAAAAAczYUJHX3A4/2gbv7XUnOOtr7BQAAAGA+hoa2VdUJVfWfq+q82fOHVNUz19oOAAAAgOUxOkfSBUk+l+SbZ88/nORn51IRAAAAAJvSaJD04O7+pSS3J0l3fyYr0+sDAAAAsEWMBkmfq6rjM7u3X1U9OMltc6sKAAAAgE1n9K5te5L8aZIHVNVrkzwu7rIGAAAAsKWM3rXt0qq6KsljszKk7SXdfdNcKwMAAABgUxm9a9ubu/ufuvv/dPcfd/dNVfXmeRcHAAAAwOZxyB5JVXWvJCckuX9V3TdfnGB7e5JT5lwbAAAAAJvIWkPbXpjkx5KcnOSqfDFI+kSSc+dYFwAAAACbzCGDpO5+RZJXVNWLuvuVG1QTAAAAAJvQ6GTbr6yqhyf5uiT3WrX8NfMqDAAAAIDNZShIqqpzkuzOSpD0J0menuTyJIIkAAAAgC1i6K5tSb4zyZOS7Ovus5M8IsmOuVUFAAAAwKYzGiTd2t1fSPL5qtqe5KNJHjC/sgAAAADYbIaGtiW5sqruk+TVWbl726eSvGNuVQEAAACw6VR3H94GVacn2d7d182joAOO1cnh1bcoduzYnVtueVuWtX0rKof79wXA8qqqeN9bXM7f4toK5077Ftnyt89ry6Ja3nOXrJy/7q71bDs0tK2q3nzn4+6+obuvW70MAAAAgOV3yKFtVXWvJCckuX9V3TcrcXGSbE9yypxrAwAAAGATWWuOpBcm+bEkJ2dlbqQ7fTLJufMqCgAAAIDNZ60g6e1Jfj/Jd3b3K6vqB5I8J8kNSS6cc23ARHbtOj379984dRlzs3Pnadm374apywAAYOkdN5tLCJbHISfbrqqrkzy5u2+uqickeV2SFyV5ZJKv7e7vnGtxJttecMs9OdkyM3EeMA9eWxab87e4tsK5075Fpn2La5nblizz+0JyZJNtr9Uj6djuvnn2+LuTnNfdFyW5qKquXc8BAQAAAFhMa9217diqujNselKSt6z63VohFAAAAABLZK0w6PeSvK2qbkpya5L/myRVdUaSW+ZcGwAAAACbyCGDpO7+uap6c5KvTnJpf3GA4DFZmSsJAAAAgC1izeFp3X3F3Sx7/3zKAQAAAGCzWmuOJAAAAABIIkgCAAAAYJAgCQAAAIAhgiQAAAAAhgiSAAAAABgiSAIAAABgiCAJAAAAgCGCJAAAAACGCJIAAAAAGCJIAgAAAGCIIAkAAACAIZMGSVV1TFVdXVUXT1kHAAAAAGubukfSS5K8Z+IaAAAAABgwWZBUVacmeUaS35yqBgAAAADGTdkj6b8n+ckkPWENAAAAAAyaJEiqqm9Lsr+7r01Ssx8AAAAANrFtEx33cUmeVVXPSHJ8kntX1Wu6+/lfvuqeVY93z34AOJhdu07P/v03Tl3G3OzceVr27bth6jKApXNcqvzbJgDLae/evdm7d+9R2Vd1TzuyrKr+ZZKXdvez7uZ3vawj33bs2J1bbnlblrV9KypT/32xPisfpJf53C3336bzx2blb3OxbYXzt7ztW+a2Jdq36LRvcS1z25Kt8L7e3ev6F5Sp79oGAAAAwIKYamjbXbr7bUneNnUdAAAAAByaHkkAAAAADBEkAQAAADBEkAQAAADAEEESAAAAAEMESQAAAAAMESQBAAAAMESQBAAAAMAQQRIAAAAAQwRJAAAAAAwRJAEAAAAwRJAEAAAAwBBBEgAAAABDBEkAAAAADBEkAQAAADBEkAQAAADAEEESAAAAAEMESQAAAAAMESQBAAAAMESQBAAAAMCQbVMXwDI7LlU1dRFzsXPnadm374apy4AtymsLAABMRZDEHN2WpKcuYi7271/OL7GwGLy2AADAVAxtAwAAAGCIIAkAAACAIYIkAAAAAIYIkgAAAAAYIkgCAAAAYIggCQAAAIAhgiQAAAAAhgiSAAAAABgiSAIAAABgiCAJAAAAgCGCJAAAAACGCJIAAAAAGCJIAgAAAGCIIAkAAACAIYIkAAAAAIYIkgAAAAAYIkgCAAAAYMgkQVJVnVpVb6mqv66qd1fVi6eoAwAAAIBx2yY67ueT/IfuvraqTkpyVVVd2t3vnageAAAAANYwSY+k7t7X3dfOHn8qyfVJTpmiFgAAAADGTD5HUlWdnuSRSf5i2koAAAAAOJSphrYlSWbD2v4gyUtmPZPuxp5Vj3fPfmBqx6Wqpi4CAACAufCd72AmC5KqaltWQqTf6e43HHzNPRtUERyO25L01EXMkRdMAABgK/Od72CmHNr2W0ne092vmLAGAAAAAAZNEiRV1eOSfG+SJ1bVNVV1dVU9bYpaAAAAABgzydC27v7zJMdOcWwAAAAA1mfyu7YBAAAAsBgESQAAAAAMESQBAAAAMESQBAAAAMAQQRIAAAAAQwRJAAAAAAwRJAEAAAAwRJAEAAAAwBBBEgAAAABDBEkAAAAADBEkAQAAADBEkAQAAADAEEESAAAAAEMESQAAAAAMESQBAAAAMESQBAAAAMAQQRIAAAAAQwRJAAAAAAwRJAEAAAAwZNvUBQBsvONSVVMXASwdry0AwPITJAFb0G1Jeuoi5sgXWZiG1xYAYPkZ2gYAAADAEEESAAAAAEMESQAAAAAMESQBAAAAMESQBAAAAMAQQRIAAAAAQwRJAAAAAAwRJAEAAAAwRJAEAAAAwBBBEgAAAABDBEkAAAAADBEkAQAAADBEkAQAAADAEEESAAAAAEMESQAAAAAMESQBAAAAMGSyIKmqnlZV762q91fVy6aqAwAAAIAxkwRJVXVMknOTPDXJ1yd5blU9bIpaAAAAABgzVY+kxyT5QHff2N23J3ldkmdPVAsAAAAAA6YKkk5J8qFVz/9htgwAAACATWrb1AWsZfv2b5+6hLn47GffPXUJAAAAAIdlqiDpw0m+ZtXzU2fLvswnPvHHG1LQdGrqAuZsmdu3zG1LtG/Rad+iqlretq3QvsWmfYtrmduWaN+i077FtcxtS5a/fetT3b3xB606Nsn7kjwpyT8meWeS53b39RteDAAAAABDJumR1N13VNWPJrk0K/M0nS9EAgAAANjcJumRBAAAAMDimequbV+iqp5WVe+tqvdX1csOss6vVtUHquraqnrkRtcIy2ita6+qnldV75r9XF5V3zBFnbBsRt73ZuudVVW3V9W/3sj6YFkNfubcXVXXVNVfVdVbN7pGWEYDnzm3V9XFs+96766qF0xQJiydqjq/qvZX1XWHWOews5bJg6SqOibJuUmemuTrkzy3qh52wDpPT/Lg7n5Ikhcm+R8bXigsmZFrL8nfJXlCdz8iyc8mefXGVgnLZ/Dau3O9X0xyycZWCMtp8DPnjiS/luSZ3f3wJN+14YXCkhl83/uRJH/d3Y9M8i1JfrmqNv0dxmEBXJCVa+9urTdrmTxISvKYJB/o7hu7+/Ykr0vy7APWeXaS1yRJd/9Fkh1VtXNjy4Sls+a1191XdPcts6dXJDllg2uEZTTyvpckL0ryB0k+upHFwRIbufael+Si7v5wknT3TRtcIyyjkWuvk9x79vjeSf6puz+/gTXCUuruy5N87BCrrCtr2QxB0ilJPrTq+T/ky7+sHrjOh+9mHeDwjFx7q/3bJG+ca0WwNax57VXVyUm+o7t/Pe47C0fLyPveQ5Pcr6reWlV/WVXfv2HVwfIaufbOTfJ1VfWRJO9K8pINqg22unVlLboLAmuqqm9JcnaSx09dC2wRv5Jk9RwSwiTYGNuSPCrJE5OcmOQdVfWO7v6bacuCpffUJNd09xOr6sFJLquqM7v7U1MXBny5zRAkfTjJ16x6fups2YHrPGCNdYDDM3LtparOTHJekqd196G6RQJjRq69f57kdVVVSe6f5OlVdXt3X7xBNcIyGrn2/iHJTd392SSfrao/S/KIJIIkWL+Ra+/sJL+QJN39t1X1wSQPS3LlhlQIW9e6spbNMLTtL5OcUVWnVdU9k3xPkgM/KF+c5PlJUlWPTfLx7t6/sWXC0lnz2quqr0lyUZLv7+6/naBGWEZrXnvd/aDZzwOzMk/SvxciwREb+cz5hiSPr6pjq+qEJN+U5PoNrhOWzci1d2OSJyfJbH6Wh2blpi/AkascvHf7urKWyXskdfcdVfWjSS7NSrB1fndfX1UvXPl1n9fdf1JVz6iqv0ny6awk1sARGLn2kvyXJPdL8qpZz4jbu/sx01UNi2/w2vuSTTa8SFhCg58531tVlyS5LskdSc7r7vdMWDYsvMH3vZ9N8j9X3aL8P3b3zROVDEujqi5MsjvJV1TV3yc5J8k9c4RZS3X7fAoAAADA2jbD0DYAAAAAFoAgCQAAAIAhgiQAAAAAhgiSAAAAABgiSAIAAABgiCAJAAAAgCGCJABg06qqO6rq6qq6tqqurKrHDmzzyY2oDQBgK9o2dQEAAIfw6e5+VJJU1VOS/GKS3Wts0/MuakpVdWx33zF1HQDA1qRHEgCwmdWqxzuS3JwkVXViVb1p1kvpXVX1rC/b8CDrVNVpVfWeqjqvqv6qqv60qo6b/e7BVXXZqh5QD5wt/4mqeuds+Tl3W2jVq2brvHv1OlV1VlX9+WzbK2Z1HVNV/3W27rVV9SOzdT9YVfebPX50Vb119vicqnpNVV2e5DWzNvzZrMYv6alVVS+rquuq6pqq+vmqelBVXbXq92esfg4AcDj0SAIANrPjq+rqJMcn2ZXkibPltyb5ju7+VFV9RZIrklx8wLafPcQ6ZyT57u7+4ar6X0mek+TCJK9N8vPdfXFV3TPJMVX1rUke0t2PqapKcnFVPb67Lz/geD/d3R+vqmOSvLmqLkryviSvS/Jd3X11VZ00q+uHk5yW5Mzu7qq6z2wfB/amWv38a5M8rrs/V1X3SvLk2eMzkvxekrOq6ulJvj3JWd19W1XdZ1bTx6vqzO6+LsnZSX5r7f/1AABfTpAEAGxmn1k1tO2xSX4nycOz0qv6F6rqCUm+kOTkqvqq7v7oqm3r7taZ/e6D3f3u2eOrkpw+C3lO7u6Lk6S7Pzc77lOSfOss0KokJyZ5SJIDg6Tvqaofysrnq11Jvm62/CPdffVsn5+a7fPJSX69u3u2/OOraj6Yi++sKck9k5xbVY9McsesniR5UpILuvu2A/Z7fpKzq+qlSb47yVmHOA4AwEEJkgCAhdDdV1TV/avq/km+Lcn9k3xjd3+hqj6Y5F4HbPK9h1jntlXr3bFq+d0FOZXkF7r71QerrapOT/LSJI/u7k9U1QVr7PNgPp8vTj1wYHs+verxjyfZ191nVtWxWemhdSgXJTknyVuTXNndHzuMmgAA7mKOJABgM7srhKmqh2Xls8s/ZWW+pI/OAqJvycowsQO3GVnnLrPeQh+qqmfPjnfPqjo+ySVJfrCqTpwtP7mqvvKAzbcn+VSST1bVziRPny1/X5JdVfXo2bYnzYKfy5K8cPY4VXXf2fofTPLo2ePnHOL/y44k/zh7/Pwkx84eX5aVnkfHr97vrIfSJUl+PckFh9gvAMAhCZIAgM3sXlV1dVVdk5V5gJ4/Gw722qzMCfSuJN+X5PpV29w5r9DIOgd6fpIXz7b58yQ7u/uyrMyf9I6qui7J65OctHqj2dxD186O8buZDXvr7tuzMpTs3Kq6NsmlSY5L8ptJPpTkulnbnjvb1c8k+dWqemdWeicdzKuSvGC27UMz663U3ZdkZR6oK2dD8V66apvXZqX31aWH2C8AwCHVbGg+AABLbDY/0vbuvtu7zgEAjDBHEgDAkquqP0zyoHzxrncAAOuiRxIAAAAAQ8yRBAAAAMAQQRIAAAAAQwRJAAAAAAwRJAEAAAAwRJAEAAAAwBBBEgAAAABD/j//WowQ0oxSqgAAAABJRU5ErkJggg==np.random.normal(size = 1000)
plt.figure(figsize=(20,5))
plt.hist(x, bins=20)
#plt.title('Balance accuracy of sample statements')
#matplotlib.rc('xtick', labelsize=20) 
#matplotlib.rc('ytick', labelsize=20) 
plt.xlabel('Balance accuracy',fontsize=20)
plt.ylabel('Statements',fontsize=20)

### Connect to blob storage

In [ ]:
blob_dal = AzureBlobStorageDataAccessLayer('devcapitecdata', 'Gt5z0EAAAm5ZETk//WBnRiQUZ9FDB/BmLmbx5ZPRtDwh6vxTDAB/XzLlM+m/epfnDYcZ8JDfMEGg1p9tBW0iFQ==', 'bank-statements', home_directory='drop/Documentum_line_files')

blob_dal.connect()
files = blob_dal.list(path='CI')

print ("Number of files: ", len(files))


In [ ]:

print(files[500].name)
print(files[501].name)


### Pdf viewer

In [ ]:
import commands
import time
import glob
pdfFolderDir = "./view_pdf&xml$txt/"
exampleNumber = 2608

import commands
import time

count = 0
for fileNum in range(len(files)):
    if files[fileNum].name[-3:]=="pdf":
        count += 1
        if count == exampleNumber:
            pdfName = files[fileNum].directory + files[fileNum].name
            print("Working on file number: "+str(count))
            # Download the pdf file
            start = time.time()
            matchString = files[fileNum].name[:-4]
            stream = StringIO()
            blob_dal.get(pdfName, stream)
            pdfDir = pdfFolderDir+"Example.pdf"
            with open (pdfDir, 'w') as tmp:
              stream.seek (0)
              shutil.copyfileobj (stream, tmp)
            print("It took ",round(time.time()-start,2)," seconds download the pdf.")
            break
print("Done")